In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
# def save_obj(obj, name):
#     with open('output_kflod/'+ name + '.pkl', 'wb') as f:
#         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# def load_obj(name):
#     with open('output_kflod/' + name + '.pkl', 'rb') as f:
#         return pickle.load(f)

In [2]:
def save_obj(obj, name):
    np.save('./output2_kflod/'+ name + '.npy', dict(tmp_dict))

def load_obj(name):
    return np.load('./output2_kflod/'+ name + '.npy').flatten()[0]

In [3]:
# train = pd.read_csv('../../tencent_ad/CJY/r2/input/train.csv')
# test = pd.read_csv('../../tencent_ad/CJY/r2/input/test2.csv')

# train.shape
# test.shape

# test['label'] = -99

# df = pd.concat([train, test])

# df.shape

# df.to_csv('df2b.csv', index=None)

(45539700, 3)

(11727304, 2)

(57267004, 3)

In [3]:
df = pd.read_csv('df2b.csv')
user = pd.read_csv('../../tencent_ad/CJY/r2/input/userFeature.csv')

df.shape
df.head()
user.shape
user.head()

(57267004, 3)

,aid,uid,label
0,627,17557009,-1
1,1502,17401944,1
2,1993,55569517,-1
3,1505,17799681,-1
4,210,10887549,-1


(44295207, 24)

,uid,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,2245869,5,2.0,10,6,1,13.0,70;86;47;76;48;28;83;9;36;11,47;24;46;32;30;72,NaN,NaN,72;92;37;116;13;47;71;8;86;28;6;42;76;55;20;46...,126370;448819;475255;685367;703384,105115;112067;86430;58695;116442,NaN,3207;976;2672;1087;6714,7407;2150;543;310;1574,NaN,NaN,NaN,3;1;2,2,2,NaN
1,14947958,1,2.0,10,7,1,539.0,70;100;76;28;6;42;71;57;17;56;36;11;96,19;13,NaN,NaN,72;33;92;116;13;71;109;8;50;86;28;6;2;42;75;20...,562294;731665;64588;207238;471398,87384;27763;105115;61655;121235,NaN,684;4313;8525;6586;7883,8525;8828;7661;4591;3618,NaN,NaN,NaN,3,1,1,1.0
2,16950131,1,1.0,0,7,1,514.0,70;100;76;106;36;11,NaN,NaN,NaN,72;33;78;79;109;124;30;50;42;19;64,517564;731496;395217;272264;780946,94093;81476;36301;235;56438,NaN,7684;5384;7318;3513;6353,6360;2611;1327;4931;7107,NaN,NaN,NaN,1;3,2,3,NaN
3,76895171,1,2.0,10,7,2,921.0,70;100;76;106;49;84;114;95;68;113;17;56;36;11,79;73;1;46;70;6;74;72,6;8;4;10;9;1;7;5;2,NaN,72;37;116;8;50;6;42;76;27;46;64;103;90;40;52;1...,308185;627369;782899;523681,29946;120667;8646;16186;39618,36983;30214;8820;36499;14542,9418;9390;2393;1834;9826,3604;3863;9399;2392;661,5015;43;7467;1042;2189,NaN,NaN,1;3,1,1,1.0
4,53838952,1,2.0,10,5,1,621.0,70;76;69;73;36;11;38;90;87,70;54,NaN,NaN,52;72;92;107;79;8;50;6;76;27;75;60;129;130;40;93,739307;117916;709037;100680;368434,50582;79577;70440,NaN,6432;5571;4275;9830;4126,466;950;5927;3436;1859,NaN,NaN,NaN,3;1,1,1,NaN


In [4]:
train = df[df.label != -99]
train['label'][train.label == -1] = 0

train.shape

(45539700, 3)

In [4]:
kflod = pd.read_csv('kflod5_auid_r2(10).csv')

kflod.shape
kflod.groupby(['kflod']).uid.count()

(45539700, 3)

kflod
0    4554723
1    4554561
2    4554393
3    4554196
4    4554032
5    4553879
6    4553719
7    4553563
8    4553408
9    4553226
Name: uid, dtype: int64

In [4]:
split_col = ['marriageStatus', 'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
            'topic1', 'topic2', 'topic3', 'appIdInstall', 'appIdAction', 'ct', 'os']
feature_col = ['age', 'gender', 'marriageStatus', 'education', 'consumptionAbility', 'LBS', 'interest1', 'interest2',
               'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3', 'topic1', 'topic2', 'topic3', 'appIdInstall',
               'appIdAction', 'ct', 'os', 'carrier']
single_col = ['age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier','house']

def spl_col(x):
    if (type(x) == float) | (type(x) == int):
        return x
    elif type(x) == list:
        return x[0].split(';')
    else:
        return x.split(';')

# history

In [5]:
from collections import defaultdict, Counter
import gc
import ast

In [8]:
for k in tqdm(range(kflod.kflod.max() + 1)):
    tmp_his = kflod[kflod.kflod == k]
    tmp_his = tmp_his.merge(train, on=['aid', 'uid'], how='left')
    tmp_his = tmp_his.merge(user, on=['uid'], how='left')
    
    for i in split_col:
        tmp_his[i] = tmp_his[i].apply(spl_col)

    tmp_his_pos = tmp_his[tmp_his.label == 1]
    
    for i in split_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in range(0, tmp_his.shape[0]):
            if type(tmp_his[i][j]) == list:
                for word in tmp_his[i][j]:
                    tmp_dict[word] += 1
        save_obj(tmp_dict, str(i + '_dict_' + str(k)))
        del tmp_dict
        _ = gc.collect()
        
    for i in split_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in list(tmp_his_pos.index):
            if type(tmp_his_pos[i][j]) == list:
                for word in tmp_his_pos[i][j]:
                    tmp_dict[word] += 1
        save_obj(tmp_dict, str(i + '_pos_dict_' + str(k)))
        del tmp_dict
        _ = gc.collect()
        
    for i in feature_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in range(0, tmp_his.shape[0]):
            if type(tmp_his[i][j]) == list:
                tmp_dict[tuple(tmp_his[i][j])] += 1
            elif (type(tmp_his[i][j]) == np.int64) | (type(tmp_his[i][j]) == np.float64):
                tmp_dict[tmp_his[i][j]] += 1
            else:
                tmp_dict[tuple([999])] += 1
        save_obj(tmp_dict, str(i + '_full_dict_' + str(k)))
        del tmp_dict
        _ = gc.collect()
        
    for i in feature_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in list(tmp_his_pos.index):
            if type(tmp_his_pos[i][j]) == list:
                tmp_dict[tuple(tmp_his_pos[i][j])] += 1
            elif (type(tmp_his_pos[i][j]) == np.int64) | (type(tmp_his_pos[i][j]) == np.float64):
                tmp_dict[tmp_his_pos[i][j]] += 1
            else:
                tmp_dict[tuple([999])] += 1
        save_obj(tmp_dict, str(i + '_full_pos_dict_' + str(k)))
        del tmp_dict
        _ = gc.collect()
    
    del tmp_his
    del tmp_his_pos
    _ = gc.collect()

100%|██████████| 10/10 [28:00:27<00:00, 10082.72s/it]  


# train

In [7]:
df.groupby(['label']).aid.count()

train = df[df.label != -99]
test = df[df.label == -99]

label
-99    11729073
-1     43357297
 1      2182403
Name: aid, dtype: int64

In [ ]:
for k in tqdm(range(kflod.kflod.max() + 1)):
    tmp_tr = kflod[kflod.kflod == k] # train
#     tmp_tr = test # test
    tmp_his = kflod[kflod.kflod != k]
    
    tmp_tr = tmp_tr.merge(train, on=['aid', 'uid'], how='left') # train
    tmp_tr = tmp_tr.merge(user, on=['uid'], how='left')
    tmp_tr.LBS = tmp_tr.LBS.fillna(tmp_tr.LBS.mean())
    tmp_tr.house = tmp_tr.house.fillna(0)
    
    tmp_his = tmp_his.merge(train, on=['aid', 'uid'], how='left')
    tmp_his = tmp_his.merge(user[single_col + ['uid']], on=['uid'], how='left')
    tmp_his.LBS = tmp_his.LBS.fillna(tmp_his.LBS.mean())
    tmp_his.house = tmp_his.house.fillna(0)
    tmp_his_pos = tmp_his[tmp_his.label == 1]
    
    for i in split_col:
        tmp_tr[i] = tmp_tr[i].apply(spl_col)
    
    li = list(range(kflod.kflod.max() + 1))
    li.remove(k)
    
    for i in split_col:
        tmp_dict = {}
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_dict_' + str(ti)))
            tmp_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(li.index(ti) + 1)])
            del locals()['tmp_dict' + str(li.index(ti) + 1)]
            _ = gc.collect()
        tmp_dict = dict(tmp_dict)
        
        tmp_pos_dict = {}
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_pos_dict_' + str(ti)))
            tmp_pos_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(li.index(ti) + 1)])
            del locals()['tmp_dict' + str(li.index(ti) + 1)]
            _ = gc.collect()
        tmp_pos_dict = dict(tmp_pos_dict)
        
        tmp_list1 = list(np.zeros(tmp_tr.shape[0]))
        tmp_list2 = list(np.zeros(tmp_tr.shape[0]))
        for kk in range(0, tmp_tr.shape[0]):
            if type(tmp_tr[i][kk]) == list:
                for j in tmp_tr[i][kk]:
                    if j in tmp_dict.keys():
                        tmp_list1[kk] = tmp_list1[kk] + tmp_dict[j]
                    if j in tmp_pos_dict.keys():
                        tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[j]
        tmp_tr[str(i + '_split_count_score')] = tmp_list1
        tmp_tr[str(i + '_split_pos_count_score')] = tmp_list2
        tmp_tr[str(i + '_split_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
        
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        _ = gc.collect()
        
    for i in feature_col:
        tmp_dict = {}
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_full_dict_' + str(ti)))
            tmp_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(li.index(ti) + 1)])
            del locals()['tmp_dict' + str(li.index(ti) + 1)]
            _ = gc.collect()
        tmp_dict = dict(tmp_dict)
        
        tmp_pos_dict = {}
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_full_pos_dict_' + str(ti)))
            tmp_pos_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(li.index(ti) + 1)])
            del locals()['tmp_dict' + str(li.index(ti) + 1)]
            _ = gc.collect()
        tmp_pos_dict = dict(tmp_pos_dict)
        
        tmp_list1 = list(np.zeros(tmp_tr.shape[0]))
        tmp_list2 = list(np.zeros(tmp_tr.shape[0]))
        for kk in range(0, tmp_tr.shape[0]):
            if type(tmp_tr[i][kk]) == list:
                if tuple(tmp_tr[i][kk]) in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tuple(tmp_tr[i][kk])]
                if tuple(tmp_tr[i][kk]) in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tuple(tmp_tr[i][kk])]
            elif (type(tmp_tr[i][kk]) == np.int64) | (type(tmp_tr[i][kk]) == np.float64):
                if tmp_tr[i][kk] in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tmp_tr[i][kk]]
                if tmp_tr[i][kk] in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tmp_tr[i][kk]]
        tmp_tr[str(i + '_feature_count_score')] = tmp_list1
        tmp_tr[str(i + '_feature_pos_count_score')] = tmp_list2
        tmp_tr[str(i + '_feature_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
        
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        _ = gc.collect()
        
    for i in single_col:
        locals()[str(i + '_feature')] = tmp_his.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_max'):
                'max',
                str(i + '_min'):
                'min',
                str(i + '_mean'):
                'mean',
                str(i + '_std'):
                'std'
            })
        locals()[str(i + '_pos_feature')] = tmp_his_pos.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_pos_max'):
                'max',
                str(i + '_pos_min'):
                'min',
                str(i + '_pos_mean'):
                'mean',
                str(i + '_pos_std'):
                'std'
            })
        tmp_tr = tmp_tr.merge(locals()[str(i + '_feature')], on=['aid'], how='left')
        tmp_tr = tmp_tr.merge(locals()[str(i + '_pos_feature')], on=['aid'], how='left')
        tmp_tr[str(i + '_mean_sub')] = abs(tmp_tr[i] - tmp_tr[str(i + '_mean')])
        tmp_tr[str(i + '_mean_pos_sub')] = abs(tmp_tr[i] - tmp_tr[str(i + '_pos_mean')])
        del locals()[str(i + '_feature')]
        del locals()[str(i + '_pos_feature')]
        _ = gc.collect()
        
    tmp_tr.to_csv('./output_r2/train_kflod_new' + str(k), index=None)
    del tmp_tr
    del tmp_his
    del tmp_his_pos
    _ = gc.collect()

 10%|█         | 1/10 [7:31:34<67:44:12, 27094.68s/it]

# test

In [9]:
test.head()

,aid,uid,label,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,1886,37963888,-99,5,2.0,"[13, 10]",1,2,772.0,"[93, 77, 109, 75, 10, 29, 6, 39, 46, 116, 36, 11]","[63, 30]",NaN,NaN,"[52, 100, 80, 131, 78, 109, 12, 8, 30, 28, 6, ...","[68211, 734509, 82594, 53354, 411962]","[57049, 66915, 106194, 111759, 79676]",NaN,"[2002, 3162, 8876, 7895, 3442]","[7278, 8265, 8384, 8763, 895]",NaN,NaN,NaN,"[3, 1]",[1],1,0.0
1,1474,58023815,-99,4,1.0,"[6, 13]",7,1,791.0,"[93, 70, 37, 109, 98, 47, 75, 1, 69, 102, 8, 2...","[73, 24, 46, 28, 30, 74, 43, 37, 29]",NaN,NaN,"[100, 92, 37, 116, 13, 47, 1, 71, 109, 8, 28, ...","[37390, 446434, 218208, 271883, 687593]","[94407, 8125, 28206, 41124, 15571]",NaN,"[6580, 9826, 7684, 9981, 3450]","[1376, 5103, 2751, 7537, 2737]",NaN,NaN,NaN,"[3, 1, 4]",[1],1,1.0
2,1039,15963481,-99,2,2.0,[10],3,1,958.0,"[75, 29, 59, 49, 36, 11]",NaN,NaN,NaN,"[116, 78, 30, 6, 42, 121, 74, 59, 64, 129]","[138321, 500100, 691107, 582619, 530883]","[73947, 34004, 61197, 16229]",NaN,"[1297, 9981, 2768, 8666, 4848]","[1078, 1647, 3739, 5651, 8967]",NaN,NaN,NaN,[3],[1],3,0.0
3,55,19888357,-99,2,2.0,[10],7,2,275.0,"[70, 76, 48, 28, 106, 49, 15, 17, 56]","[51, 73, 79, 1, 46, 70, 6, 19, 4, 36, 18, 13, ...",NaN,NaN,"[72, 92, 101, 134, 47, 78, 71, 109, 8, 30, 50,...","[363004, 325756, 102493, 162804, 10251]","[52864, 461, 12933, 93916, 117462]",NaN,"[1400, 4392, 7684, 1552, 2672]","[3305, 900, 3313, 5641, 7314]",NaN,NaN,NaN,"[3, 1]",[2],2,1.0
4,863,65666195,-99,2,1.0,[10],6,0,222.0,NaN,NaN,NaN,NaN,NaN,NaN,"[3725, 8656, 23419, 66797, 15760]",NaN,NaN,"[4362, 5499, 9118, 1423, 4107]",NaN,NaN,NaN,[0],[0],0,0.0


In [8]:
test = df[df.label == -99]
test.shape
test['kflod'] = 0

test['kflod'][:int(0.25 * test.shape[0])] = 0
test['kflod'][int(0.25 * test.shape[0]):int(0.5 * test.shape[0])] = 1
test['kflod'][int(0.5 * test.shape[0]):int(0.75 * test.shape[0])] = 2
test['kflod'][int(0.75 * test.shape[0]):] = 3

test.groupby(['kflod']).uid.count()

(11727304, 3)

kflod
0    2931826
1    2931826
2    2931826
3    2931826
Name: uid, dtype: int64

In [9]:
test.to_csv('test_kflodcut_b.csv', index=None)

In [6]:
test_kflod = pd.read_csv('test_kflodcut_b.csv')
    
tmp_his = df[df.label != -99]
tmp_his = tmp_his.merge(user[single_col + ['uid']], on=['uid'], how='left')
tmp_his.LBS = tmp_his.LBS.fillna(tmp_his.LBS.mean())
tmp_his.house = tmp_his.house.fillna(0)
tmp_his_pos = tmp_his[tmp_his.label == 1]
    
del df
_ = gc.collect()

for ii in tqdm(range(4)):
    test = test_kflod[test_kflod.kflod == ii]
    test = test.merge(user, on=['uid'], how='left')
    test.LBS = test.LBS.fillna(test.LBS.mean())
    test.house = test.house.fillna(0)
        
    for i in split_col:
        test[i] = test[i].apply(spl_col)
    
    for i in split_col:
        tmp_dict = {}
        for ti in range(0, 10):
            locals()['tmp_dict' + str(ti)] = load_obj(str(i + '_dict_' + str(ti)))
            tmp_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(ti)])
            del locals()['tmp_dict' + str(ti)]
            _ = gc.collect()
        tmp_dict = dict(tmp_dict)
        
        tmp_pos_dict = {}
        for ti in range(0, 10):
            locals()['tmp_dict' + str(ti)] = load_obj(str(i + '_pos_dict_' + str(ti)))
            tmp_pos_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(ti)])
            del locals()['tmp_dict' + str(ti)]
            _ = gc.collect()
        tmp_pos_dict = dict(tmp_pos_dict)
            
        tmp_list1 = list(np.zeros(test.shape[0]))
        tmp_list2 = list(np.zeros(test.shape[0]))
        for kk in range(0, test.shape[0]):
            if type(test[i][kk]) == list:
                for j in test[i][kk]:
                    if j in test.keys():
                        tmp_list1[kk] = tmp_list1[kk] + tmp_dict[j]
                    if j in tmp_pos_dict.keys():
                        tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[j]
        test[str(i + '_split_count_score')] = (np.array(tmp_list1) * 9 / 10).astype(int)
        test[str(i + '_split_pos_count_score')] = (np.array(tmp_list2) * 9 / 10).astype(int)
        test[str(i + '_split_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
            
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        _ = gc.collect()
        
    for i in feature_col:
        tmp_dict = {}
        for ti in range(0, 10):
            locals()['tmp_dict' + str(ti)] = load_obj(str(i + '_full_dict_' + str(ti)))
            tmp_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(ti)])
            del locals()['tmp_dict' + str(ti)]
            _ = gc.collect()
        tmp_dict = dict(tmp_dict)
        
        tmp_pos_dict = {}
        for ti in range(0, 10):
            locals()['tmp_dict' + str(ti)] = load_obj(str(i + '_full_pos_dict_' + str(ti)))
            tmp_pos_dict = Counter(tmp_dict) + Counter(locals()['tmp_dict' + str(ti)])
            del locals()['tmp_dict' + str(ti)]
            _ = gc.collect()
        tmp_pos_dict = dict(tmp_pos_dict)
            
        tmp_list1 = list(np.zeros(test.shape[0]))
        tmp_list2 = list(np.zeros(test.shape[0]))
        for kk in range(0, test.shape[0]):
            if type(test[i][kk]) == list:
                if tuple(test[i][kk]) in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tuple(test[i][kk])]
                if tuple(test[i][kk]) in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tuple(test[i][kk])]
            elif (type(test[i][kk]) == np.int64) | (type(test[i][kk]) == np.float64):
                if test[i][kk] in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[test[i][kk]]
                if test[i][kk] in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[test[i][kk]]
        test[str(i + '_feature_count_score')] = (np.array(tmp_list1) * 9 / 10).astype(int)
        test[str(i + '_feature_pos_count_score')] = (np.array(tmp_list2) * 9 / 10).astype(int)
        test[str(i + '_feature_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
            
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        _ = gc.collect()
        
    for i in single_col:
        locals()[str(i + '_feature')] = tmp_his.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_max'):
                'max',
                str(i + '_min'):
                'min',
                str(i + '_mean'):
                'mean',
                str(i + '_std'):
                'std'
            })
        locals()[str(i + '_pos_feature')] = tmp_his_pos.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_pos_max'):
                'max',
                str(i + '_pos_min'):
                'min',
                str(i + '_pos_mean'):
                'mean',
                str(i + '_pos_std'):
                'std'
            })
        test = test.merge(locals()[str(i + '_feature')], on=['aid'], how='left')
        test = test.merge(locals()[str(i + '_pos_feature')], on=['aid'], how='left')
        test[str(i + '_mean_sub')] = abs(test[i] - test[str(i + '_mean')])
        test[str(i + '_mean_pos_sub')] = abs(test[i] - test[str(i + '_pos_mean')])
        del locals()[str(i + '_feature')]
        del locals()[str(i + '_pos_feature')]
        _ = gc.collect()
    
    test.to_csv('./output_r2/test_kflod_new_b' + str(ii), index=None)
    del test
    _ = gc.collect()

100%|██████████| 4/4 [21:23:27<00:00, 19251.92s/it]   


In [ ]:
# test = df[df.label == -99]
# test.reset_index(drop=True, inplace=True)
# test = test.merge(user, on=['uid'], how='left')
# test.LBS = test.LBS.fillna(test.LBS.mean())
# test.house = test.house.fillna(0)
    
# for i in split_col:
#     test[i] = test[i].apply(spl_col)

# tmp_his = df[df.label != -99]
# tmp_his = tmp_his.merge(user[single_col + ['uid']], on=['uid'], how='left')
# tmp_his.LBS = tmp_his.LBS.fillna(tmp_his.LBS.mean())
# tmp_his.house = tmp_his.house.fillna(0)
# tmp_his_pos = tmp_his[tmp_his.label == 1]

# del df
# gc.collect()

# for i in split_col:
#     for ti in range(0, 10):
#         locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_dict_' + str(ti)))
#     tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
#     tmp_dict = dict(tmp_dict)
    
#     for ti in range(0, 10):
#         locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_pos_dict_' + str(ti)))
#     tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
#     tmp_pos_dict = dict(tmp_pos_dict)
        
#     del tmp_dict1
#     del tmp_dict2
#     del tmp_dict3
#     del tmp_dict4
#     del tmp_dict5
#     gc.collect()
        
#     tmp_list1 = list(np.zeros(test.shape[0]))
#     tmp_list2 = list(np.zeros(test.shape[0]))
#     for kk in range(0, test.shape[0]):
#         if type(test[i][kk]) == list:
#             for j in test[i][kk]:
#                 if j in test.keys():
#                     tmp_list1[kk] = tmp_list1[kk] + tmp_dict[j]
#                 if j in tmp_pos_dict.keys():
#                     tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[j]
#     test[str(i + '_split_count_score')] = (np.array(tmp_list1) * 9 / 10).astype(int)
#     test[str(i + '_split_pos_count_score')] = (np.array(tmp_list2) * 9 / 10).astype(int)
#     test[str(i + '_split_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
        
#     del tmp_dict
#     del tmp_pos_dict
#     del tmp_list1
#     del tmp_list2
#     gc.collect()
    
# for i in feature_col:
#     for ti in range(0, 10):
#         locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_full_dict_' + str(ti)))
#     tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
#     tmp_dict = dict(tmp_dict)
    
#     for ti in range(0, 10):
#         locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_full_pos_dict_' + str(ti)))
#     tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
#     tmp_pos_dict = dict(tmp_pos_dict)
        
#     del tmp_dict1
#     del tmp_dict2
#     del tmp_dict3
#     del tmp_dict4
#     del tmp_dict5
#     gc.collect()
        
#     tmp_list1 = list(np.zeros(test.shape[0]))
#     tmp_list2 = list(np.zeros(test.shape[0]))
#     for kk in range(0, test.shape[0]):
#         if type(test[i][kk]) == list:
#             if tuple(test[i][kk]) in tmp_dict.keys():
#                 tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tuple(test[i][kk])]
#             if tuple(test[i][kk]) in tmp_pos_dict.keys():
#                 tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tuple(test[i][kk])]
#         elif (type(test[i][kk]) == np.int64) | (type(test[i][kk]) == np.float64):
#             if test[i][kk] in tmp_dict.keys():
#                 tmp_list1[kk] = tmp_list1[kk] + tmp_dict[test[i][kk]]
#             if test[i][kk] in tmp_pos_dict.keys():
#                 tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[test[i][kk]]
#     test[str(i + '_feature_count_score')] = (np.array(tmp_list1) * 9 / 10).astype(int)
#     test[str(i + '_feature_pos_count_score')] = (np.array(tmp_list2) * 9 / 10).astype(int)
#     test[str(i + '_feature_pos_percent')] = list(np.true_divide(tmp_list2, tmp_list1 + np.ones(len(tmp_list1))))
        
#     del tmp_dict
#     del tmp_pos_dict
#     del tmp_list1
#     del tmp_list2
#     gc.collect()
    
# for i in single_col:
#     locals()[str(i + '_feature')] = tmp_his.groupby(
#         ['aid'], as_index=False)[i].agg({
#             str(i + '_max'):
#             'max',
#             str(i + '_min'):
#             'min',
#             str(i + '_mean'):
#             'mean',
#             str(i + '_std'):
#             'std'
#         })
#     locals()[str(i + '_pos_feature')] = tmp_his_pos.groupby(
#         ['aid'], as_index=False)[i].agg({
#             str(i + '_pos_max'):
#             'max',
#             str(i + '_pos_min'):
#             'min',
#             str(i + '_pos_mean'):
#             'mean',
#             str(i + '_pos_std'):
#             'std'
#         })
#     test = test.merge(locals()[str(i + '_feature')], on=['aid'], how='left')
#     test = test.merge(locals()[str(i + '_pos_feature')], on=['aid'], how='left')
#     test[str(i + '_mean_sub')] = abs(test[i] - test[str(i + '_mean')])
#     test[str(i + '_mean_pos_sub')] = abs(test[i] - test[str(i + '_pos_mean')])
#     del locals()[str(i + '_feature')]
#     del locals()[str(i + '_pos_feature')]
#     gc.collect()

# test.to_csv('./output_r2/test_kflod', index=None)
# del test
# del tmp_his
# del tmp_his_pos
# gc.collect()

46

60

21

60

21

...

21

60

21

60

21

# xgb分开

In [3]:
import xgboost as xgb
import gc

In [4]:
full_col = ['carrier_pos_max','carrier_min','gender_pos_min','interest1_feature_count_score','carrier','carrier_feature_count_score','kw1_feature_pos_count_score','interest2_feature_count_score','appIdAction_split_pos_count_score','education_max','ct_feature_count_score','interest2_feature_pos_count_score','topic3_feature_count_score','house_pos_mean','LBS_mean_sub','education','kw1_split_pos_count_score','os_split_pos_count_score','appIdInstall_split_pos_count_score','interest4_split_count_score','interest5_split_pos_count_score','LBS_pos_mean','topic2_feature_pos_count_score','education_std','carrier_std','consumptionAbility_pos_std','gender_std','interest2_split_pos_count_score','age_std','kw2_split_pos_count_score','gender_mean_sub','LBS_pos_std','topic3_split_pos_count_score','consumptionAbility_max','carrier_max','LBS_feature_count_score','consumptionAbility_mean','house_pos_max','appIdAction_feature_pos_count_score','LBS_min','consumptionAbility_mean_pos_sub','consumptionAbility_feature_count_score','house_mean','age_mean','house_pos_min','house_pos_std','LBS_mean_pos_sub','interest1_feature_pos_count_score','ct_split_count_score','topic2_split_pos_count_score','topic2_feature_count_score','topic1_split_count_score','age_feature_pos_count_score','aid','interest3_split_count_score','kw3_split_count_score','house_max','interest5_feature_count_score','kw2_feature_count_score','LBS_std','house_mean_sub','LBS_mean','kw3_split_pos_count_score','topic1_split_pos_count_score','interest1_split_pos_count_score','gender','gender_pos_mean','gender_feature_count_score','appIdAction_feature_count_score','carrier_mean_pos_sub','interest3_split_pos_count_score','age_pos_min','education_pos_mean','house_std','age_mean_sub','marriageStatus_feature_count_score','appIdInstall_feature_pos_count_score','house','education_pos_max','consumptionAbility_pos_min','age_feature_count_score','education_mean_pos_sub','interest3_feature_count_score','age_mean_pos_sub','gender_pos_max','uid','kw2_split_count_score','topic2_split_count_score','education_feature_count_score','interest4_feature_pos_count_score','age_max','carrier_pos_min','gender_feature_pos_count_score','consumptionAbility_std','education_pos_min','interest4_split_pos_count_score','consumptionAbility_pos_mean','interest5_split_count_score','os_feature_count_score','os_split_count_score','label','appIdInstall_split_count_score','LBS_pos_max','kw3_feature_count_score','topic3_split_count_score','topic1_feature_pos_count_score','education_pos_std','LBS_pos_min','carrier_pos_mean','topic1_feature_count_score','education_min','carrier_mean','education_mean','kw2_feature_pos_count_score','consumptionAbility_pos_max','consumptionAbility_mean_sub','gender_pos_std','age_min','interest4_feature_count_score','ct_split_pos_count_score','LBS','gender_min','marriageStatus_split_pos_count_score','gender_mean_pos_sub','carrier_feature_pos_count_score','topic3_feature_pos_count_score','interest5_feature_pos_count_score','age_pos_max','appIdAction_split_count_score','LBS_max','kw1_feature_count_score','marriageStatus_split_count_score','education_mean_sub','consumptionAbility_min','marriageStatus_feature_pos_count_score','consumptionAbility_feature_pos_count_score','appIdInstall_feature_count_score','LBS_feature_pos_count_score','interest3_feature_pos_count_score','age','os_feature_pos_count_score','interest1_split_count_score','kw1_split_count_score','kw3_feature_pos_count_score','ct_feature_pos_count_score','age_pos_std','carrier_pos_std','gender_mean','gender_max','age_pos_mean','house_mean_pos_sub','house_min','education_feature_pos_count_score','consumptionAbility','interest2_split_count_score','carrier_mean_sub']

In [ ]:
# full_col = list(set(train1.columns) - set(split_col) - set(['is_his', 'kflod']))
predictors = list(set(train1.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
target = ['label']
len(set(predictors) & set(train1.columns))
# len(set(predictors) & set(test.columns))
len(predictors)

In [11]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': 0
}

len(predictors)

train1['label'][train1.label == -1] = 0
xgbtrain = xgb.DMatrix(train1[predictors], train1[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=10000,
    nfold=3,
    verbose_eval=100,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-auc-mean']))
# 1980:0.72182

1980:0.72182


In [6]:
test = pd.read_csv('test_kflod', usecols=['aid', 'uid'])

test.shape
test.head()

(2265989, 2)

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [14]:
#train/test
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
#     'scale_pos_weight': int(train[train.label == 0].shape[0]/train[train.label == 1].shape[0]),
    'verbose_eval': 1,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

# for i in range(0, 5):
#     tmp_train = pd.read_csv('tr_kflod' + str(i), usecols=full_col)
#     tmp_test = pd.read_csv('test_kflod' + str(i), usecols=full_col)
#     predictors = list(set(tmp_train.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
#     len(predictors)
#     target = ['label']
#     print('start kflod_' + str(i))
#     tmp_train['label'][tmp_train.label == -1] = 0
#     xgbtrain = xgb.DMatrix(tmp_train[predictors], tmp_train[target])
#     xgbtest = xgb.DMatrix(tmp_test[predictors])

#     model = xgb.train(params, xgbtrain, num_boost_round=1980, verbose_eval=100)
#     y_pred = model.predict(xgbtest)
#     y_pred = pd.Series(y_pred, name='test')
    
#     test['score_' + str(i)] = list(y_pred)
#     print('finish kflod_' + str(i))
    
#     del tmp_train
#     del tmp_test
#     gc.collect()
    
test['score'] = (test['score_0'] + test['score_1'] + test['score_2'] + test['score_3'] + test['score_4']) / 5

In [15]:
test[['aid','uid','score_0','score_1','score_2','score_3','score_4','score']].head()

,aid,uid,score_0,score_1,score_2,score_3,score_4,score
0,2118,64355836,0.046616,0.047110,0.042533,0.043293,0.048661,0.045643
1,692,45051997,0.029959,0.031260,0.037797,0.040466,0.032353,0.034367
2,692,10869198,0.034535,0.040661,0.027282,0.032500,0.028964,0.032788
3,1918,75929554,0.031406,0.038289,0.031384,0.042147,0.035950,0.035835
4,1596,5790162,0.138514,0.214496,0.153955,0.207861,0.194549,0.181875


In [16]:
import os
result = test[['aid','uid','score_1']]
result.columns = ['aid','uid','score']
result.to_csv('submission.csv', index=False, float_format='%.8f')
os.system('zip ./result/xgb_kflod_new.zip submission.csv')

0

# xgb合体

In [2]:
import xgboost as xgb
import gc

In [3]:
full_col = ['carrier_pos_max','carrier_min','gender_pos_min','interest1_feature_count_score','carrier','carrier_feature_count_score','kw1_feature_pos_count_score','interest2_feature_count_score','appIdAction_split_pos_count_score','education_max','ct_feature_count_score','interest2_feature_pos_count_score','topic3_feature_count_score','house_pos_mean','LBS_mean_sub','education','kw1_split_pos_count_score','os_split_pos_count_score','appIdInstall_split_pos_count_score','interest4_split_count_score','interest5_split_pos_count_score','LBS_pos_mean','topic2_feature_pos_count_score','education_std','carrier_std','consumptionAbility_pos_std','gender_std','interest2_split_pos_count_score','age_std','kw2_split_pos_count_score','gender_mean_sub','LBS_pos_std','topic3_split_pos_count_score','consumptionAbility_max','carrier_max','LBS_feature_count_score','consumptionAbility_mean','house_pos_max','appIdAction_feature_pos_count_score','LBS_min','consumptionAbility_mean_pos_sub','consumptionAbility_feature_count_score','house_mean','age_mean','house_pos_min','house_pos_std','LBS_mean_pos_sub','interest1_feature_pos_count_score','ct_split_count_score','topic2_split_pos_count_score','topic2_feature_count_score','topic1_split_count_score','age_feature_pos_count_score','aid','interest3_split_count_score','kw3_split_count_score','house_max','interest5_feature_count_score','kw2_feature_count_score','LBS_std','house_mean_sub','LBS_mean','kw3_split_pos_count_score','topic1_split_pos_count_score','interest1_split_pos_count_score','gender','gender_pos_mean','gender_feature_count_score','appIdAction_feature_count_score','carrier_mean_pos_sub','interest3_split_pos_count_score','age_pos_min','education_pos_mean','house_std','age_mean_sub','marriageStatus_feature_count_score','appIdInstall_feature_pos_count_score','house','education_pos_max','consumptionAbility_pos_min','age_feature_count_score','education_mean_pos_sub','interest3_feature_count_score','age_mean_pos_sub','gender_pos_max','uid','kw2_split_count_score','topic2_split_count_score','education_feature_count_score','interest4_feature_pos_count_score','age_max','carrier_pos_min','gender_feature_pos_count_score','consumptionAbility_std','education_pos_min','interest4_split_pos_count_score','consumptionAbility_pos_mean','interest5_split_count_score','os_feature_count_score','os_split_count_score','label','appIdInstall_split_count_score','LBS_pos_max','kw3_feature_count_score','topic3_split_count_score','topic1_feature_pos_count_score','education_pos_std','LBS_pos_min','carrier_pos_mean','topic1_feature_count_score','education_min','carrier_mean','education_mean','kw2_feature_pos_count_score','consumptionAbility_pos_max','consumptionAbility_mean_sub','gender_pos_std','age_min','interest4_feature_count_score','ct_split_pos_count_score','LBS','gender_min','marriageStatus_split_pos_count_score','gender_mean_pos_sub','carrier_feature_pos_count_score','topic3_feature_pos_count_score','interest5_feature_pos_count_score','age_pos_max','appIdAction_split_count_score','LBS_max','kw1_feature_count_score','marriageStatus_split_count_score','education_mean_sub','consumptionAbility_min','marriageStatus_feature_pos_count_score','consumptionAbility_feature_pos_count_score','appIdInstall_feature_count_score','LBS_feature_pos_count_score','interest3_feature_pos_count_score','age','os_feature_pos_count_score','interest1_split_count_score','kw1_split_count_score','kw3_feature_pos_count_score','ct_feature_pos_count_score','age_pos_std','carrier_pos_std','gender_mean','gender_max','age_pos_mean','house_mean_pos_sub','house_min','education_feature_pos_count_score','consumptionAbility','interest2_split_count_score','carrier_mean_sub']

In [4]:
# train1 = pd.read_csv('tr_kflod0', usecols=full_col)
# train2 = pd.read_csv('tr_kflod1', usecols=full_col)
# train3 = pd.read_csv('tr_kflod2', usecols=full_col)
# train4 = pd.read_csv('tr_kflod3', usecols=full_col)
# train5 = pd.read_csv('tr_kflod4', usecols=full_col)
test1 = pd.read_csv('test_kflod_b0', usecols=full_col)
test2 = pd.read_csv('test_kflod_b1', usecols=full_col)
test3 = pd.read_csv('test_kflod_b2', usecols=full_col)
test4 = pd.read_csv('test_kflod_b3', usecols=full_col)
test5 = pd.read_csv('test_kflod_b4', usecols=full_col)


# train1.shape
# train2.shape
# train3.shape
# train4.shape
# train5.shape
test1.shape
test2.shape
test3.shape
test4.shape
test5.shape

FileNotFoundError: File b'test_kflod0_b' does not exist

In [5]:
train = pd.concat([train1, train2, train3, train4, train5])

del train1
del train2
del train3
del train4
del train5

gc.collect()

28

In [8]:
test = test1 + test2 + test3 + test4 + test5
test = test / 5

test1.head()
test.head()

,aid,uid,label,age,gender,education,consumptionAbility,LBS,carrier,house,marriageStatus_split_count_score,marriageStatus_split_pos_count_score,interest1_split_count_score,interest1_split_pos_count_score,interest2_split_count_score,interest2_split_pos_count_score,interest3_split_count_score,interest3_split_pos_count_score,interest4_split_count_score,interest4_split_pos_count_score,interest5_split_count_score,interest5_split_pos_count_score,kw1_split_count_score,kw1_split_pos_count_score,kw2_split_count_score,kw2_split_pos_count_score,kw3_split_count_score,kw3_split_pos_count_score,topic1_split_count_score,topic1_split_pos_count_score,topic2_split_count_score,topic2_split_pos_count_score,topic3_split_count_score,topic3_split_pos_count_score,appIdInstall_split_count_score,appIdInstall_split_pos_count_score,appIdAction_split_count_score,appIdAction_split_pos_count_score,ct_split_count_score,ct_split_pos_count_score,os_split_count_score,os_split_pos_count_score,age_feature_count_score,age_feature_pos_count_score,gender_feature_count_score,gender_feature_pos_count_score,marriageStatus_feature_count_score,marriageStatus_feature_pos_count_score,education_feature_count_score,education_feature_pos_count_score,consumptionAbility_feature_count_score,consumptionAbility_feature_pos_count_score,LBS_feature_count_score,LBS_feature_pos_count_score,interest1_feature_count_score,interest1_feature_pos_count_score,interest2_feature_count_score,interest2_feature_pos_count_score,interest3_feature_count_score,interest3_feature_pos_count_score,interest4_feature_count_score,interest4_feature_pos_count_score,interest5_feature_count_score,interest5_feature_pos_count_score,kw1_feature_count_score,kw1_feature_pos_count_score,kw2_feature_count_score,kw2_feature_pos_count_score,kw3_feature_count_score,kw3_feature_pos_count_score,topic1_feature_count_score,topic1_feature_pos_count_score,topic2_feature_count_score,topic2_feature_pos_count_score,topic3_feature_count_score,topic3_feature_pos_count_score,appIdInstall_feature_count_score,appIdInstall_feature_pos_count_score,appIdAction_feature_count_score,appIdAction_feature_pos_count_score,ct_feature_count_score,ct_feature_pos_count_score,os_feature_count_score,os_feature_pos_count_score,carrier_feature_count_score,carrier_feature_pos_count_score,age_max,age_min,age_mean,age_std,age_pos_max,age_pos_min,age_pos_mean,age_pos_std,age_mean_sub,age_mean_pos_sub,gender_max,gender_min,gender_mean,gender_std,gender_pos_max,gender_pos_min,gender_pos_mean,gender_pos_std,gender_mean_sub,gender_mean_pos_sub,education_max,education_min,education_mean,education_std,education_pos_max,education_pos_min,education_pos_mean,education_pos_std,education_mean_sub,education_mean_pos_sub,consumptionAbility_max,consumptionAbility_min,consumptionAbility_mean,consumptionAbility_std,consumptionAbility_pos_max,consumptionAbility_pos_min,consumptionAbility_pos_mean,consumptionAbility_pos_std,consumptionAbility_mean_sub,consumptionAbility_mean_pos_sub,LBS_max,LBS_min,LBS_mean,LBS_std,LBS_pos_max,LBS_pos_min,LBS_pos_mean,LBS_pos_std,LBS_mean_sub,LBS_mean_pos_sub,carrier_max,carrier_min,carrier_mean,carrier_std,carrier_pos_max,carrier_pos_min,carrier_pos_mean,carrier_pos_std,carrier_mean_sub,carrier_mean_pos_sub,house_max,house_min,house_mean,house_std,house_pos_max,house_pos_min,house_pos_mean,house_pos_std,house_mean_sub,house_mean_pos_sub
0,2118,64355836,-99,1,1,6,2,484.0,1,0.0,5247330.0,278444.0,34998286.0,1581380.0,14816346.0,872874.0,0.0,0.0,0.0,0.0,22851678.0,1023735.0,261786.0,12325.0,71456.0,3087.0,0.0,0.0,45977.0,2227.0,112060.0,10284.0,0.0,0.0,0.0,0.0,0.0,0.0,11776394.0,536647.0,4690694.0,231317.0,1652136.0,71958.0,4587739.0,227293.0,923054.0,56154.0,1214906.0,55527.0,1365943.0,82224.0,19853.0,887.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160800.0,6945.0,4163659.0,209714.0,3394348.0,155795.0,5,0,2.214488,1.348491,5,1,2.143225,1.133780,1.214488,1.143225,2,0,1.282551,0.4515

,aid,uid,label,age,gender,education,consumptionAbility,LBS,carrier,house,marriageStatus_split_count_score,marriageStatus_split_pos_count_score,interest1_split_count_score,interest1_split_pos_count_score,interest2_split_count_score,interest2_split_pos_count_score,interest3_split_count_score,interest3_split_pos_count_score,interest4_split_count_score,interest4_split_pos_count_score,interest5_split_count_score,interest5_split_pos_count_score,kw1_split_count_score,kw1_split_pos_count_score,kw2_split_count_score,kw2_split_pos_count_score,kw3_split_count_score,kw3_split_pos_count_score,topic1_split_count_score,topic1_split_pos_count_score,topic2_split_count_score,topic2_split_pos_count_score,topic3_split_count_score,topic3_split_pos_count_score,appIdInstall_split_count_score,appIdInstall_split_pos_count_score,appIdAction_split_count_score,appIdAction_split_pos_count_score,ct_split_count_score,ct_split_pos_count_score,os_split_count_score,os_split_pos_count_score,age_feature_count_score,age_feature_pos_count_score,gender_feature_count_score,gender_feature_pos_count_score,marriageStatus_feature_count_score,marriageStatus_feature_pos_count_score,education_feature_count_score,education_feature_pos_count_score,consumptionAbility_feature_count_score,consumptionAbility_feature_pos_count_score,LBS_feature_count_score,LBS_feature_pos_count_score,interest1_feature_count_score,interest1_feature_pos_count_score,interest2_feature_count_score,interest2_feature_pos_count_score,interest3_feature_count_score,interest3_feature_pos_count_score,interest4_feature_count_score,interest4_feature_pos_count_score,interest5_feature_count_score,interest5_feature_pos_count_score,kw1_feature_count_score,kw1_feature_pos_count_score,kw2_feature_count_score,kw2_feature_pos_count_score,kw3_feature_count_score,kw3_feature_pos_count_score,topic1_feature_count_score,topic1_feature_pos_count_score,topic2_feature_count_score,topic2_feature_pos_count_score,topic3_feature_count_score,topic3_feature_pos_count_score,appIdInstall_feature_count_score,appIdInstall_feature_pos_count_score,appIdAction_feature_count_score,appIdAction_feature_pos_count_score,ct_feature_count_score,ct_feature_pos_count_score,os_feature_count_score,os_feature_pos_count_score,carrier_feature_count_score,carrier_feature_pos_count_score,age_max,age_min,age_mean,age_std,age_pos_max,age_pos_min,age_pos_mean,age_pos_std,age_mean_sub,age_mean_pos_sub,gender_max,gender_min,gender_mean,gender_std,gender_pos_max,gender_pos_min,gender_pos_mean,gender_pos_std,gender_mean_sub,gender_mean_pos_sub,education_max,education_min,education_mean,education_std,education_pos_max,education_pos_min,education_pos_mean,education_pos_std,education_mean_sub,education_mean_pos_sub,consumptionAbility_max,consumptionAbility_min,consumptionAbility_mean,consumptionAbility_std,consumptionAbility_pos_max,consumptionAbility_pos_min,consumptionAbility_pos_mean,consumptionAbility_pos_std,consumptionAbility_mean_sub,consumptionAbility_mean_pos_sub,LBS_max,LBS_min,LBS_mean,LBS_std,LBS_pos_max,LBS_pos_min,LBS_pos_mean,LBS_pos_std,LBS_mean_sub,LBS_mean_pos_sub,carrier_max,carrier_min,carrier_mean,carrier_std,carrier_pos_max,carrier_pos_min,carrier_pos_mean,carrier_pos_std,carrier_mean_sub,carrier_mean_pos_sub,house_max,house_min,house_mean,house_std,house_pos_max,house_pos_min,house_pos_mean,house_pos_std,house_mean_sub,house_mean_pos_sub
0,2118.0,64355836.0,-99.0,1.0,1.0,6.0,2.0,484.0,1.0,0.0,5247593.6,278500.8,34996520.8,1580804.0,14820950.4,873020.0,0.0,0.0,0.0,0.0,22852395.2,1024552.0,261404.0,12252.0,71291.2,3116.8,0.0,0.0,45887.2,2268.0,111974.4,10283.2,0.0,0.0,0.0,0.0,0.0,0.0,11775762.4,536634.4,4690014.4,231332.0,1652848.0,71979.2,4587864.0,227326.4,923299.2,56146.4,1215188.8,55593.6,1364797.6,82197.6,19875.2,888.8,0.0,0.0,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160652.8,6895.2,4162556.0,209678.4,3394492.0,155712.8,5.0,0.0,2.217602,1.350347,5.0,1.0,2.146810,1.138495,1.217602,1.14

In [10]:
# train.to_csv('train_kflod.csv', index=None)
test.to_csv('test_kflod_new.csv', index=None)

In [7]:
# full_col = list(set(train1.columns) - set(split_col) - set(['is_his', 'kflod']))
predictors = list(set(train.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
target = ['label']
len(set(predictors) & set(train.columns))
# len(set(predictors) & set(test.columns))
len(predictors)

153

153

In [ ]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': 0
}

len(predictors)

# train['label'][train.label == -1] = 0
# xgbtrain = xgb.DMatrix(train[predictors], train[target])
print('start training')

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=10000,
    nfold=3,
    verbose_eval=100,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-auc-mean']))

153

start training
[0]	train-auc:0.622387+0.000371669	test-auc:0.622323+0.000362091


In [6]:
test = pd.read_csv('test_kflod', usecols=['aid', 'uid'])

test.shape
test.head()

(2265989, 2)

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [ ]:
#train/test
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
#     'scale_pos_weight': int(train[train.label == 0].shape[0]/train[train.label == 1].shape[0]),
    'verbose_eval': 1,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

len(predictors)

train['label'][train.label == -1] = 0
xgbtrain = xgb.DMatrix(train[predictors], train[target])
xgbtest = xgb.DMatrix(tmp_test[predictors])

model = xgb.train(params, xgbtrain, num_boost_round=1980, verbose_eval=100)
y_pred = model.predict(xgbtest)
y_pred = pd.Series(y_pred, name='test')

In [16]:
test.head()

,aid,uid,label,is_his,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house,marriageStatus_split_count_score,marriageStatus_split_pos_count_score,interest1_split_count_score,interest1_split_pos_count_score,interest2_split_count_score,interest2_split_pos_count_score,interest3_split_count_score,interest3_split_pos_count_score,interest4_split_count_score,interest4_split_pos_count_score,interest5_split_count_score,interest5_split_pos_count_score,kw1_split_count_score,kw1_split_pos_count_score,kw2_split_count_score,kw2_split_pos_count_score,kw3_split_count_score,kw3_split_pos_count_score,topic1_split_count_score,topic1_split_pos_count_score,topic2_split_count_score,topic2_split_pos_count_score,topic3_split_count_score,topic3_split_pos_count_score,appIdInstall_split_count_score,appIdInstall_split_pos_count_score,appIdAction_split_count_score,appIdAction_split_pos_count_score,ct_split_count_score,ct_split_pos_count_score,os_split_count_score,os_split_pos_count_score,age_feature_count_score,age_feature_pos_count_score,gender_feature_count_score,gender_feature_pos_count_score,marriageStatus_feature_count_score,marriageStatus_feature_pos_count_score,education_feature_count_score,education_feature_pos_count_score,consumptionAbility_feature_count_score,consumptionAbility_feature_pos_count_score,LBS_feature_count_score,LBS_feature_pos_count_score,interest1_feature_count_score,interest1_feature_pos_count_score,interest2_feature_count_score,interest2_feature_pos_count_score,interest3_feature_count_score,interest3_feature_pos_count_score,interest4_feature_count_score,interest4_feature_pos_count_score,interest5_feature_count_score,interest5_feature_pos_count_score,kw1_feature_count_score,kw1_feature_pos_count_score,kw2_feature_count_score,kw2_feature_pos_count_score,kw3_feature_count_score,kw3_feature_pos_count_score,topic1_feature_count_score,topic1_feature_pos_count_score,topic2_feature_count_score,topic2_feature_pos_count_score,topic3_feature_count_score,topic3_feature_pos_count_score,appIdInstall_feature_count_score,appIdInstall_feature_pos_count_score,appIdAction_feature_count_score,appIdAction_feature_pos_count_score,ct_feature_count_score,ct_feature_pos_count_score,os_feature_count_score,os_feature_pos_count_score,carrier_feature_count_score,carrier_feature_pos_count_score,age_max,age_min,age_mean,age_std,age_pos_max,age_pos_min,age_pos_mean,age_pos_std,age_mean_sub,age_mean_pos_sub,gender_max,gender_min,gender_mean,gender_std,gender_pos_max,gender_pos_min,gender_pos_mean,gender_pos_std,gender_mean_sub,gender_mean_pos_sub,education_max,education_min,education_mean,education_std,education_pos_max,education_pos_min,education_pos_mean,education_pos_std,education_mean_sub,education_mean_pos_sub,consumptionAbility_max,consumptionAbility_min,consumptionAbility_mean,consumptionAbility_std,consumptionAbility_pos_max,consumptionAbility_pos_min,consumptionAbility_pos_mean,consumptionAbility_pos_std,consumptionAbility_mean_sub,consumptionAbility_mean_pos_sub,LBS_max,LBS_min,LBS_mean,LBS_std,LBS_pos_max,LBS_pos_min,LBS_pos_mean,LBS_pos_std,LBS_mean_sub,LBS_mean_pos_sub,carrier_max,carrier_min,carrier_mean,carrier_std,carrier_pos_max,carrier_pos_min,carrier_pos_mean,carrier_pos_std,carrier_mean_sub,carrier_mean_pos_sub,house_max,house_min,house_mean,house_std,house_pos_max,house_pos_min,house_pos_mean,house_pos_std,house_mean_sub,house_mean_pos_sub,score_1,score_2,score_3,score_4,score_5,score
0,2118,64355836,-99,-99.0,1,1,"['13', '10']",6,2,484.0,"['70', '86', '100', '76', '75', '28', '106', '...","['47', '22', '24', '73', '9', '1', '46', '32',...",NaN,NaN,"['72', '37', '13', '79', '6', '46', '94', '59'...","['89154', '196312', '627369', '87552', '440482']","['119567', '104708', '61089', '4272', '90835']",NaN,"['9163', '1390', '9359', '7441', '2266']","['8582', '1471', '679', '4552', '845']",NaN,NaN,NaN,"['1', '3', '4